<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/A/Depthmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1444407462603063334, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16907001087352125846
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2336238729101258375
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17517689786308229956
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [2]:
!git clone https://github.com/roshantac/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 5), reused 47 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [3]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-07 17:46:35--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.84.2
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.84.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  86.7MB/s    in 1.9s    

2020-05-07 17:46:37 (86.7 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp '/content/gdrive/My Drive/dataSet2.zip' '/content/DenseDepth/'

In [6]:
%cd DenseDepth

/content/DenseDepth


In [0]:
mkdir 'Data'

In [0]:
destDataDir = '/content/DenseDepth/Data/'
path_to_zip_file = '/content/DenseDepth/dataSet2.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
rm '/content/DenseDepth/dataSet2.zip'

In [0]:
import skimage
from skimage.transform import resize
import os
import shutil
import numpy as np
from PIL import Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt


try:
  os.mkdir("/content/DenseDepth/Data/DepthMap")
except:
  print("Either folder already exists or tree is not exist")

srcpath = "/content/DenseDepth/Data/Output/fgbg/"
destpath = "/content/DenseDepth/Data/DepthMap/" 
prrMdl = 'nyu.h5'
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}
print('Loading model...')
model = load_model(prrMdl, custom_objects=custom_objects, compile=False)
print('loaded ...')
for idx,d in enumerate(os.listdir(srcpath)):
  loaded_images = []
  imgsrc = srcpath+ d
  print("imgsrc",imgsrc)
  img = np.clip(np.asarray((Image.open(imgsrc)).resize((640,480),Image.BILINEAR), dtype=float) / 255, 0, 1)
  outputs = predict(model, img)
  print(outputs.shape)
  rescaled = outputs[0][:,:,0]
  print("rscaled ",rescaled.shape ,"  out: ",outputs[0].shape)
  rescaled = rescaled - np.min(rescaled)
  rescaled = rescaled / np.max(rescaled)
  im = Image.fromarray(255-np.uint8(rescaled*255))
  im.save(destpath + d, optimize=True, quality=60)
  print(idx)

Using TensorFlow backend.


Streaming output truncated to the last 5000 lines.
rscaled  (240, 320)   out:  (240, 320, 1)
144863
imgsrc /content/DenseDepth/Data/Output/fgbg/img79554.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144864
imgsrc /content/DenseDepth/Data/Output/fgbg/img134109.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144865
imgsrc /content/DenseDepth/Data/Output/fgbg/img384235.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144866
imgsrc /content/DenseDepth/Data/Output/fgbg/img222393.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144867
imgsrc /content/DenseDepth/Data/Output/fgbg/img300388.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144868
imgsrc /content/DenseDepth/Data/Output/fgbg/img250833.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144869
imgsrc /content/DenseDepth/Data/Output/fgbg/img205405.jpg
(1, 240, 320, 1)
rscaled  (240, 320)   out:  (240, 320, 1)
144870
imgsrc /content/DenseDepth/Data/O

In [0]:
# importing required modules 
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = '/content/DenseDepth/Data/DepthMap'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('depthMap.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')  
main()

he


In [0]:
!cp '/content/DenseDepth/depthMap.zip' '/content/gdrive/My Drive/'